In [1]:
import pandas as pd
from lib import standardize_item_numbers
import numpy as np

In [2]:
def clean():
    dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")
    dfa = dfa.pipe(standardize_item_numbers, ["item_number"])

    dfb = pd.read_csv("../data/real_time_crime_center/rtcc.csv")
    dfb["rtcc_requested"] = "1"
    dfb = dfb.pipe(standardize_item_numbers, ["item_number"])

    df = pd.merge(dfa, dfb, on="item_number")
    return dfa

In [3]:
df = clean()

C:\Users\ayyubi\AppData\Local\Temp\ipykernel_73600\1424771964.py:2: DtypeWarning: Columns (13,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")


In [4]:
df.columns

Index(['item_number', 'district', 'location', 'disposition', 'signal_type',
       'signal_description', 'occurred_date_time', 'charge_code',
       'charge_description', 'offender_race', 'offender_gender',
       'offender_age', 'offender_number', 'person_type', 'victim_race',
       'victim_gender', 'victim_age', 'victim_number', 'victim_fatal_status',
       'hate_crime', 'report_type', 'year', 'offenderid', 'offenderstatus',
       'persontype'],
      dtype='object')

In [5]:
df.loc[:, "location"] = df.location.str.replace(r"(\w+)$", r"\1, New Orleans, Louisiana")

requested = df[["item_number", "location"]]

requested_1 = requested[:5000]
requested_2 = requested[5000:10000]
requested_3 = requested[10000:15000]
requested_4 = requested[15000:]

# requested_1.to_csv("../data/real_time_crime_center/rtcc_locations_1.csv", index=False)
# requested_2.to_csv("../data/real_time_crime_center/rtcc_locations_2.csv", index=False)
# requested_3.to_csv("../data/real_time_crime_center/rtcc_locations_3.csv", index=False)
# requested_4.to_csv("../data/real_time_crime_center/rtcc_locations_4.csv", index=False)

C:\Users\ayyubi\AppData\Local\Temp\ipykernel_73600\1998154791.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "location"] = df.location.str.replace(r"(\w+)$", r"\1, New Orleans, Louisiana")


In [6]:
requested_1_coordinates = pd.read_csv("../data/real_time_crime_center/rtcc_locations_1_coordinates.csv")
requested_1_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_1.csv")
rtcc_1 = pd.concat([requested_1_df, requested_1_coordinates], axis=1)

In [7]:
requested_2_coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_2_coordinates.csv")
requested_2_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_2.csv")
rtcc_2 = pd.concat([requested_2_df, requested_2_coordinates_df], axis=1)

In [8]:
requested_3_coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_3_coordinates.csv")
requested_3_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_3.csv")
rtcc_3 = pd.concat([requested_3_df, requested_3_coordinates_df], axis=1)

In [9]:
requested_4_coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_4_coordinates.csv")
requested_4_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_4.csv")
rtcc_4 = pd.concat([requested_4_df, requested_4_coordinates_df], axis=1)

In [10]:
coordinates_df = pd.concat([rtcc_1, rtcc_2, rtcc_3, rtcc_4], axis=0)

In [11]:
# coordinates_df.to_csv("../data/real_time_crime_center/rtcc_coordinates.csv", index=False)

In [12]:
coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_coordinates.csv")
coordinates_df = coordinates_df.rename(columns={"0": "latitude", "1": "longitude"})
coordinates_df = coordinates_df[["item_number", "latitude", "longitude"]]

In [13]:
df = pd.merge(df, coordinates_df, on="item_number")

In [14]:
df = df.drop_duplicates()

In [15]:
df['occurred_date_time'] = sorted(pd.to_datetime(df['occurred_date_time']))
df['hour'] = df['occurred_date_time'].dt.hour
df = df[~((df.hour.fillna("") == ""))]
df.loc[:, "hour"] = df.hour.astype(str)

In [16]:
df.disposition.unique()

array(['OPEN', 'CLOSED', 'UNFOUNDED'], dtype=object)

In [18]:
df

,item_number,district,location,disposition,signal_type,signal_description,occurred_date_time,charge_code,charge_description,offender_race,...,victim_fatal_status,hate_crime,report_type,year,offenderid,offenderstatus,persontype,latitude,longitude,hour
0,k1922422,5,"Louisa St & Abundance St, New Orleans, Louisiana",OPEN,95G,ILLEGAL CARRYING OF A GUN,2018-01-08 16:30:00,14 95,ILLEGAL CARRYING OF WEAPON,BLACK,...,Non-fatal,NaN,Incident Report,2022,789879.0,ARRESTED,VICTIM,29.987869,-90.036258,16
1,k1775022,8,"002XX Bourbon St, New Orleans, Louisiana",OPEN,21,MISCELLANEOUS INCIDENT,2018-01-08 16:30:00,NaN,NaN,NaN,...,Non-fatal,NaN,Incident Report,2022,NaN,NaN,INTERVIEW,29.954712,-90.069155,16
3,k1700722,8,"Canal St & Bourbon St, New Orleans, Louisiana",CLOSED,35,SIMPLE BATTERY,2018-01-26 02:00:00,54 96,BATTERY,WHITE,...,Non-fatal,NaN,Incident Report,2022,789591.0,ARRESTED,VICTIM,29.953871,-90.069899,2
7,k1700722,8,"Canal St & Bourbon St, New Orleans, Louisiana",CLOSED,35,SIMPLE BATTERY,2018-01-26 02:00:00,NaN,NaN,WHITE,...,Non-fatal,NaN,Incident Report,2022,789591.0,ARRESTED,WITNESS,29.953871,-90.069899,2
11,k1700722,8,"Canal St & Bourbon St, New Orleans, Louisiana",CLOSED,35,SIMPLE BATTERY,2018-01-26 02:00:00,NaN,NaN,WHITE,...,Non-fatal,NaN,Incident Report,2022,789591.0,ARRESTED,VICTIM,29.953871,-90.069899,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102896,a3354319,8,"5XX Bourbon St, New Orleans, Louisiana",OPEN,35,SIMPLE BATTERY,2022-11-18 02:10:00,54 96,BATTERY,BLACK,...,Non-fatal,NaN,Incident Report,2018,NaN,NaN,NaN,29.954712,-90.069155,2
102900,a3354319,8,"5XX Bourbon St, New Orleans, Louisiana",OPEN,35,SIMPLE BATTERY,2022-11-18 02:10:00,54 96,BATTERY,BLACK,...,Non-fatal,NaN,Incident Report,2018,NaN,NaN,NaN,29.954712,-90.069155,2
102904,a3354319,8,"5XX Bourbon St, New Orleans, Louisiana",OPEN,35,SIMPLE BATTERY,2022-11-18 02:10:00,54 96,BATTERY,BLACK,...,Non-fatal,NaN,Incident Report,2018,NaN,NaN,NaN,29.954712,-90.069155,2
102908,a1011019,1,"15XX Iberville St, New Orleans, Louisiana",CLOSED,966,DRUG LAW VIOLATION,2022-11-18 19:15:00,40 967,PROHIBITED ACTS-- SCHEDULE II DRUGS,WHITE,...,Non-fatal,NaN,Supplemental Report,2018,NaN,NaN,NaN,29.966554,-90.086616,19


In [17]:
# dates = df[["occurred_date_time", "latitude", "longitude", "hour", "item_number", "disposition"]]
# dates = dates[~((dates.occurred_date_time.fillna("") == ""))]

# def split_dates(df):
#     data = df.occurred_date_time.astype(str).str.extract(r"^(\w+-\w+-\w+) (.+)")
#     df.loc[:, "occurred_date"] = data[0]
#     df.loc[:, "occurred_time"] = data[1]    
#     return df

# dates = dates.pipe(split_dates)
# dates.to_csv("../data/real_time_crime_center/dates_6.csv", index=False)
# dates

# night = dates[dates.hour.isin(["19", "20", "21", "22", "23", "00", "1", "2", "3", "4", "5", "6"])]
# night["night"]  = "night"

# day = dates[~((dates.hour.isin(["19", "20", "21", "22", "23", "00", "1", "2", "3", "4", "5", "6"])))]
# day["day"] = 1


# night.to_csv("../data/real_time_crime_center/night.csv", index=False)
# day.to_csv("../data/real_time_crime_center/day.csv", index=False)